### IMPORT LIBRARY

In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate_update import *
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from termcolor import colored
from send_email import *

### CONNECT TO AZURE SQL

In [2]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

#Test Connection
try:
    result = engine.execute("SELECT 1")
    print("CONNECTION SUCESSFUL!")
except Exception as e:
    print("CONNECTION FAILED:",str(e))

### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
config_BIHub
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Internal Hub _ Successful Connection!


### READ AND LIST FOLDER

In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Binh Duong Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Dong Nai Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Long An Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Hung Yen Province


In [5]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Binh Duong Province/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Dong Nai Province/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Long An Province/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Hung Yen Province/2022


In [6]:
#lấy quý
list_folder_sub2=[]
for i in list_folder_sub1:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub2=list_folder_sub2+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Binh Duong Province/2022/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Dong Nai Province/2022/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2022/Q3
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2022/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022/Q3
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Long An Province/2022/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Hung Yen Province/2022/Q3


In [11]:
#Provide information
Province_name=['Binh Duong Province']#Chọn tỉnh cần import-- là tên folder 
Year=['2023']#Chọn năm cần import
Quarter=['Q1']#Chọn quý cần import
#Có thể chọn nhiều tỉnh, tên tỉnh để trong dấu nháy đơn và cách nhau bởi dấy phẩy. Áp dụng tương tự cho năm, quý
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub2:
    if i.split('/')[-3] in Province_name and i.split('/')[-2] in Year and i.split('/')[-1] in Quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_url = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2022/Q4/Update_SA_HCM_20221231.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2022/Q4/Update_Retail_HCM_20221231.xlsx
Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022/Q4/Update_Office_HN_20221231.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022/Q4/Update_SA_HN_20221231.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022/Q4/Update_Apt_HN_20221231.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022/Q4/Update_Hotel_HN_20221231.xlsx


### GET IMPORT FILE

In [8]:
url = []
for i in list_url:
    sector = i.split('/')[-1].split('_')[1].upper()
    if sector in ('RETAIL'):#Chọn sector cần import: RETAIL, OFFICE, SA, HOTEL, APT, VLTH
        url.append(i)
    else:
        pass
url

['/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2022/Q4/Update_Retail_HCM_20221231.xlsx']

### IMPORT DATA

In [9]:
'''Prepare ingredients''' 
columns_that_need_unidecode = ['Sub_Project_Name', 'Project_District_Name', 'Project_City_Name'
                               , 'New_Sub_Project_Name', 'New_Project_District_Name', 'New_Project_City_Name'
                              ]
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
name_sector = ['RETAIL', 'SA', 'APT', 'VLTH', 'OFFICE', 'HOTEL']
for i in name_sector:
    globals()[f'df_update_{i.lower()}'] = pd.DataFrame([])
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):  
    data, file_name, sector = get_data(relative_url, file_url)
    #-------------------------------------------------------
    '''Validation step'''
    #Fill up if lười điền =))
    data['New_Sub_Project_Name']= np.where(data['New_Sub_Project_Name'].isnull(), data['Sub_Project_Name'], data['New_Sub_Project_Name'])
    #Remove unfortmated values
    data = remove_unformated_character(data)
    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    lst_dict = ['City', 'District', 'Grade', 'Type']
    for p in lst_dict:
        lst_p = [x for x in data.columns[data.columns.str.contains(p)]]
        for i, j in zip([x for x in data[lst_p] if 'New' not in x], [x for x in data[lst_p] if 'New' in x]):
            data[j] = data[j].fillna(data[i])
        for i in lst_p:
            data, df_dict = check_dictionary(df_dict, file_name, data, i, p, sector, engine, sp_object)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        #-------------------------------------------------------
        '''Import data process'''
        #Check project key
        processed_data, flag_key = check_project_key(file_url, data, sector, engine)
        new_processed_data, flag_new_key = check_new_key(file_url, data, sector, engine)
        if flag_key == 0 and flag_new_key ==0:
            data = get_project_key(flag_key, processed_data, data, sector, engine)
            data = get_new_key(flag_new_key, new_processed_data, data, sector, engine)
            if sector == 'RETAIL':
                df_update_retail = pd.concat([df_update_retail, data], axis=0)
            elif sector == 'OFFICE':
                df_update_office = pd.concat([df_update_office, data], axis=0)
            elif sector == 'HOTEL':
                df_update_hotel = pd.concat([df_update_hotel, data], axis=0)
            elif sector == 'SA' or sector=='SERVICED_APARTMENT':
                df_update_sa = pd.concat([df_update_sa, data], axis=0)
            elif sector == 'APT' or sector=='APARTMENT':
                df_update_apt = pd.concat([df_update_apt, data], axis=0)
            elif sector == 'VLTH':
                df_update_vlth = pd.concat([df_update_vlth, data], axis=0)
        else:
            if flag_key != 0:
                print(colored(f'Cant get key in {file_name}', 'yellow'))
            elif flag_new_key != 0:
                print(colored(f'Cant get new key in {file_name}', 'yellow'))


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Validate succesfully


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.41s/it]


In [10]:
df_update_retail

,Update_Date,Sector,Sub_Project_Name,Project_District_Name,Project_City_Name,Sub_Project_Type,New_Sub_Project_Name,New_Project_District_Name,New_Project_City_Name,New_Sub_Project_Type,Project_Key,New_Project_Key
0,20221231,RETAIL,Big C An Lac,Binh Tan District,Ho Chi Minh City,Supermarket,Go! An Lac,Binh Tan District,Ho Chi Minh City,Supermarket,20142,45310
1,20221231,RETAIL,Big C Go Vap,Go Vap District,Ho Chi Minh City,Supermarket,Go! Go Vap,Go Vap District,Ho Chi Minh City,Supermarket,20145,45313
2,20221231,RETAIL,Big C Phu Thanh,Tan Phu District,Ho Chi Minh City,Supermarket,Go! Phu Thanh,Tan Phu District,Ho Chi Minh City,Supermarket,20147,45314
3,20221231,RETAIL,Big C An Phu,District 2,Ho Chi Minh City,Supermarket,Tops Market An Phu,District 2,Ho Chi Minh City,Supermarket,20259,45316
4,20221231,RETAIL,Big C City Land,District 7,Ho Chi Minh City,Shopping Center,Go! Nguyen Thi Thap,District 7,Ho Chi Minh City,Shopping Center,20144,45312
5,20221231,RETAIL,Big C Thao Dien Pearl,District 2,Ho Chi Minh City,Supermarket,Tops Market Thao Dien Pearl,District 2,Ho Chi Minh City,Supermarket,20149,45315
6,20221231,RETAIL,Big C Auco,Tan Phu District,Ho Chi Minh City,Supermarket,Tops Market Auco,Tan Phu District,Ho Chi Minh City,Supermarket,20143,45311
7,20221231,RETAIL,Long Son Building Phase 2022,District 7,Ho Chi Minh City,Retail Podium,Long Son Building Next Phase,District 7,Ho Chi Minh City,Retail Podium,34026,45318
8,20221231,RETAIL,Hung Vuong Plaza - Phase 2023,District 5,Ho Chi Minh City,Shopping Center,Hung Vuong Plaza Next Phase,District 5,Ho Chi Minh City,Department Store,44710,45319
9,20221231,RETAIL,Union Square Phase H22022,District 1,Ho Chi Minh City,Shopping Center,Union Square Next Phase,District 1,Ho Chi Minh City,Shopping Center,34028,45321
